In [ ]:

from transformers import GPTNeoXForCausalLM 
import pickle
from datasets import load_dataset
import torch 


In [ ]:
checkpoint_dataset = load_dataset("rdiehlmartinez/pythia-pile-presampled", "checkpoints", split="train", num_proc=8) 

In [ ]:

model_checkpoint = GPTNeoXForCausalLM.from_pretrained(
    f"EleutherAI/pythia-70m-deduped",
    revision=f"step1000",
    cache_dir=f"./pythia-70m-deduped/step1000",
).to('cuda')

In [ ]:
# Initial constants

checkpoint_dataset = load_dataset("rdiehlmartinez/pythia-pile-presampled", "checkpoints", split='train', num_proc=8)
batch = checkpoint_dataset[:30]['ids']

In [ ]:
torch.cuda.memory._record_memory_history(enabled='all')
inputs = torch.tensor(batch, device='cuda')
labels = inputs.detach().clone()
print("memory: ", torch.cuda.memory_allocated()/1e9, "GB")
outputs = model_checkpoint(input_ids=inputs)#, labels=labels)
print("memory: ", torch.cuda.memory_allocated()/1e9, "GB")
outputs['loss'].backward()
print("memory: ", torch.cuda.memory_allocated()/1e9, "GB")
model_checkpoint.zero_grad
with open('memory_history_no_labels.pkl', 'wb') as f:
    pickle.dump(torch.cuda.memory._snapshot(), f)